In [31]:
symbol_dict = [['BTC', 'USDT'], ['ETH', 'BTC'], ['ETH', 'USDT'],\
               ['BNB','USDT'],['BNB', 'BTC'],['BNB', 'ETH']
              ]
coin_dict = {'USDT':0, 'BTC':1, 'ETH':2, 'BNB':3}

N = len(coin_dict)

In [2]:
from math import log

# fee = .075 / 100

fee = .1/100

def arbitrage(table):
    transformed_graph = [[-(log(edge) + log(1.0 - fee)) for edge in row] for row in table]

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and
    # get the right result
    source = 0
    n = len(transformed_graph)
    min_dist = [float('inf')] * n

    min_dist[source] = 0

    # Relax edges |V - 1| times
    for i in range(n - 1):
        for v in range(n):
            for w in range(n):
                if min_dist[w] > min_dist[v] + transformed_graph[v][w]:
                    min_dist[w] = min_dist[v] + transformed_graph[v][w]

    # If we can still relax edges, then we have a negative cycle
    for v in range(n):
        for w in range(n):
            if min_dist[w] > min_dist[v] + transformed_graph[v][w]:
                return True

    return False

In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
market = 'binance'
timewindow = '1h'

In [5]:
data_path = '/home/nam/data/ccxt/'

In [6]:
# Load all csv data in list of Dataframe

In [7]:
from __future__ import division

In [8]:
N = len(coin_dict)
list_np = [[ None for y in range(N)] for x in range(N)]

for A, B in symbol_dict:
    print(A, B)
    
    home_path = os.path.expanduser("~")

    file_name = '{}/data/ccxt/{}_{}_{}_{}.csv'.format(home_path, A, B, market, timewindow)
    if not os.path.exists(file_name):
        continue
    
    df = pd.read_csv(file_name)
    df['Price'] = df.Close
    print(df.shape)
    
    A_idx = coin_dict.get(A)
    B_idx = coin_dict.get(B)
    
    list_np[A_idx][B_idx] = df[['Timestamp', 'Price']].values

    list_np[B_idx][A_idx] = np.copy(list_np[A_idx][B_idx])
    list_np[B_idx][A_idx][:,1] = 1.0 / list_np[A_idx][B_idx][:,1]

BTC USDT
(15000, 11)
ETH BTC
(15000, 11)
ETH USDT
(15000, 11)
BNB USDT
(15000, 11)
BNB BTC
(15000, 11)
BNB ETH
(15000, 11)


In [9]:
list_np

[[None, array([[1.51476480e+12, 7.39152384e-05],
         [1.51476840e+12, 7.57400178e-05],
         [1.51477200e+12, 7.50177417e-05],
         ...,
         [1.56906000e+12, 1.00057633e-04],
         [1.56906360e+12, 9.95365578e-05],
         [1.56906720e+12, 1.00203313e-04]]), array([[1.51476480e+12, 1.37434375e-03],
         [1.51476840e+12, 1.39281586e-03],
         [1.51477200e+12, 1.38112009e-03],
         ...,
         [1.56906000e+12, 4.65137913e-03],
         [1.56906360e+12, 4.62256737e-03],
         [1.56906720e+12, 4.63499421e-03]]), array([[1.51476480e+12, 1.19675918e-01],
         [1.51476840e+12, 1.21655981e-01],
         [1.51477200e+12, 1.21602724e-01],
         ...,
         [1.56906000e+12, 4.76917207e-02],
         [1.56906360e+12, 4.73628372e-02],
         [1.56906720e+12, 4.77981770e-02]])],
 [array([[1.5147648e+12, 1.3529010e+04],
         [1.5147684e+12, 1.3203060e+04],
         [1.5147720e+12, 1.3330180e+04],
         ...,
         [1.5690600e+12, 9.9942400e+03

In [10]:
# def load_weight(list_df, ts)

In [11]:
table = [[ None for y in range(N)] for x in range(N)]

ts = 1514764800000

for row in range(N):
    for col in range(N):
        if row == col or list_np[row][col] is None:
            
            table[row][col] = float('inf')
            continue
        row_idx = np.where(list_np[row][col][:, 0] == ts)[0][0]
        value_ratio = list_np[row][col][row_idx, 1]
        table[row][col] = value_ratio

In [12]:
table = [[ None for y in range(N)] for x in range(N)]

def search_ts_to_table(ts, list_np = list_np):
    global table
    for row in range(N):
        for col in range(N):
            if row == col or list_np[row][col] is None:

#                 table[row][col] = float('inf')
                table[row][col] = 1.0
                continue
#             assert np.isin(ts, list_np[row][col][:, 0]), print('timestamp not found')
            if not np.isin(ts, list_np[row][col][:, 0]):
#                 print('timestamp not found: {}'.format(ts))
                return False
            row_idx = np.where(list_np[row][col][:, 0] == ts)[0][0]
            value_ratio = list_np[row][col][row_idx, 1]
            table[row][col] = value_ratio
            
    return True

In [13]:
np_0 = list_np[0][1]
total_neg_cycle = 0

time_cycle_arbitrage = []
for t in np_0[:,0]:
    print(t)
    if search_ts_to_table(t, list_np=list_np):
        rs = arbitrage(table)
        if rs:
            time_cycle_arbitrage.append(t)
        total_neg_cycle += rs

print(total_neg_cycle)
print(time_cycle_arbitrage)

1514764800000.0
1514768400000.0
1514772000000.0
1514775600000.0
1514779200000.0
1514782800000.0
1514786400000.0
1514790000000.0
1514793600000.0
1514797200000.0
1514800800000.0
1514804400000.0
1514808000000.0
1514811600000.0
1514815200000.0
1514818800000.0
1514822400000.0
1514826000000.0
1514829600000.0
1514833200000.0
1514836800000.0
1514840400000.0
1514844000000.0
1514847600000.0
1514851200000.0
1514854800000.0
1514858400000.0
1514862000000.0
1514865600000.0
1514869200000.0
1514872800000.0
1514876400000.0
1514880000000.0
1514883600000.0
1514887200000.0
1514890800000.0
1514894400000.0
1514898000000.0
1514901600000.0
1514905200000.0
1514908800000.0
1514912400000.0
1514916000000.0
1514919600000.0
1514923200000.0
1514926800000.0
1514930400000.0
1514934000000.0
1514937600000.0
1514941200000.0
1514944800000.0
1514948400000.0
1514952000000.0
1514955600000.0
1514959200000.0
1514962800000.0
1514966400000.0
1514970000000.0
1514973600000.0
1514977200000.0
1514980800000.0
1514984400000.0
15149880

1516662000000.0
1516665600000.0
1516669200000.0
1516672800000.0
1516676400000.0
1516680000000.0
1516683600000.0
1516687200000.0
1516690800000.0
1516694400000.0
1516698000000.0
1516701600000.0
1516705200000.0
1516708800000.0
1516712400000.0
1516716000000.0
1516719600000.0
1516723200000.0
1516726800000.0
1516730400000.0
1516734000000.0
1516737600000.0
1516741200000.0
1516744800000.0
1516748400000.0
1516752000000.0
1516755600000.0
1516759200000.0
1516762800000.0
1516766400000.0
1516770000000.0
1516773600000.0
1516777200000.0
1516780800000.0
1516784400000.0
1516788000000.0
1516791600000.0
1516795200000.0
1516798800000.0
1516802400000.0
1516806000000.0
1516809600000.0
1516813200000.0
1516816800000.0
1516820400000.0
1516824000000.0
1516827600000.0
1516831200000.0
1516834800000.0
1516838400000.0
1516842000000.0
1516845600000.0
1516849200000.0
1516852800000.0
1516856400000.0
1516860000000.0
1516863600000.0
1516867200000.0
1516870800000.0
1516874400000.0
1516878000000.0
1516881600000.0
15168852

1518624000000.0
1518627600000.0
1518631200000.0
1518634800000.0
1518638400000.0
1518642000000.0
1518645600000.0
1518649200000.0
1518652800000.0
1518656400000.0
1518660000000.0
1518663600000.0
1518667200000.0
1518670800000.0
1518674400000.0
1518678000000.0
1518681600000.0
1518685200000.0
1518688800000.0
1518692400000.0
1518696000000.0
1518699600000.0
1518703200000.0
1518706800000.0
1518710400000.0
1518714000000.0
1518717600000.0
1518721200000.0
1518724800000.0
1518728400000.0
1518732000000.0
1518735600000.0
1518739200000.0
1518742800000.0
1518746400000.0
1518750000000.0
1518753600000.0
1518757200000.0
1518760800000.0
1518764400000.0
1518768000000.0
1518771600000.0
1518775200000.0
1518778800000.0
1518782400000.0
1518786000000.0
1518789600000.0
1518793200000.0
1518796800000.0
1518800400000.0
1518804000000.0
1518807600000.0
1518811200000.0
1518814800000.0
1518818400000.0
1518822000000.0
1518825600000.0
1518829200000.0
1518832800000.0
1518836400000.0
1518840000000.0
1518843600000.0
15188472

1520514000000.0
1520517600000.0
1520521200000.0
1520524800000.0
1520528400000.0
1520532000000.0
1520535600000.0
1520539200000.0
1520542800000.0
1520546400000.0
1520550000000.0
1520553600000.0
1520557200000.0
1520560800000.0
1520564400000.0
1520568000000.0
1520571600000.0
1520575200000.0
1520578800000.0
1520582400000.0
1520586000000.0
1520589600000.0
1520593200000.0
1520596800000.0
1520600400000.0
1520604000000.0
1520607600000.0
1520611200000.0
1520614800000.0
1520618400000.0
1520622000000.0
1520625600000.0
1520629200000.0
1520632800000.0
1520636400000.0
1520640000000.0
1520643600000.0
1520647200000.0
1520650800000.0
1520654400000.0
1520658000000.0
1520661600000.0
1520665200000.0
1520668800000.0
1520672400000.0
1520676000000.0
1520679600000.0
1520683200000.0
1520686800000.0
1520690400000.0
1520694000000.0
1520697600000.0
1520701200000.0
1520704800000.0
1520708400000.0
1520712000000.0
1520715600000.0
1520719200000.0
1520722800000.0
1520726400000.0
1520730000000.0
1520733600000.0
15207372

1522414800000.0
1522418400000.0
1522422000000.0
1522425600000.0
1522429200000.0
1522432800000.0
1522436400000.0
1522440000000.0
1522443600000.0
1522447200000.0
1522450800000.0
1522454400000.0
1522458000000.0
1522461600000.0
1522465200000.0
1522468800000.0
1522472400000.0
1522476000000.0
1522479600000.0
1522483200000.0
1522486800000.0
1522490400000.0
1522494000000.0
1522497600000.0
1522501200000.0
1522504800000.0
1522508400000.0
1522512000000.0
1522515600000.0
1522519200000.0
1522522800000.0
1522526400000.0
1522530000000.0
1522533600000.0
1522537200000.0
1522540800000.0
1522544400000.0
1522548000000.0
1522551600000.0
1522555200000.0
1522558800000.0
1522562400000.0
1522566000000.0
1522569600000.0
1522573200000.0
1522576800000.0
1522580400000.0
1522584000000.0
1522587600000.0
1522591200000.0
1522594800000.0
1522598400000.0
1522602000000.0
1522605600000.0
1522609200000.0
1522612800000.0
1522616400000.0
1522620000000.0
1522623600000.0
1522627200000.0
1522630800000.0
1522634400000.0
15226380

...

1560747600000.0
1560751200000.0
1560754800000.0
1560758400000.0
1560762000000.0
1560765600000.0
1560769200000.0
1560772800000.0
1560776400000.0
1560780000000.0
1560783600000.0
1560787200000.0
1560790800000.0
1560794400000.0
1560798000000.0
1560801600000.0
1560805200000.0
1560808800000.0
1560812400000.0
1560816000000.0
1560819600000.0
1560823200000.0
1560826800000.0
1560830400000.0
1560834000000.0
1560837600000.0
1560841200000.0
1560844800000.0
1560848400000.0
1560852000000.0
1560855600000.0
1560859200000.0
1560862800000.0
1560866400000.0
1560870000000.0
1560873600000.0
1560877200000.0
1560880800000.0
1560884400000.0
1560888000000.0
1560891600000.0
1560895200000.0
1560898800000.0
1560902400000.0
1560906000000.0
1560909600000.0
1560913200000.0
1560916800000.0
1560920400000.0
1560924000000.0
1560927600000.0
1560931200000.0
1560934800000.0
1560938400000.0
1560942000000.0
1560945600000.0
1560949200000.0
1560952800000.0
1560956400000.0
1560960000000.0
1560963600000.0
1560967200000.0
15609708

1562648400000.0
1562652000000.0
1562655600000.0
1562659200000.0
1562662800000.0
1562666400000.0
1562670000000.0
1562673600000.0
1562677200000.0
1562680800000.0
1562684400000.0
1562688000000.0
1562691600000.0
1562695200000.0
1562698800000.0
1562702400000.0
1562706000000.0
1562709600000.0
1562713200000.0
1562716800000.0
1562720400000.0
1562724000000.0
1562727600000.0
1562731200000.0
1562734800000.0
1562738400000.0
1562742000000.0
1562745600000.0
1562749200000.0
1562752800000.0
1562756400000.0
1562760000000.0
1562763600000.0
1562767200000.0
1562770800000.0
1562774400000.0
1562778000000.0
1562781600000.0
1562785200000.0
1562788800000.0
1562792400000.0
1562796000000.0
1562799600000.0
1562803200000.0
1562806800000.0
1562810400000.0
1562814000000.0
1562817600000.0
1562821200000.0
1562824800000.0
1562828400000.0
1562832000000.0
1562835600000.0
1562839200000.0
1562842800000.0
1562846400000.0
1562850000000.0
1562853600000.0
1562857200000.0
1562860800000.0
1562864400000.0
1562868000000.0
15628716

1564542000000.0
1564545600000.0
1564549200000.0
1564552800000.0
1564556400000.0
1564560000000.0
1564563600000.0
1564567200000.0
1564570800000.0
1564574400000.0
1564578000000.0
1564581600000.0
1564585200000.0
1564588800000.0
1564592400000.0
1564596000000.0
1564599600000.0
1564603200000.0
1564606800000.0
1564610400000.0
1564614000000.0
1564617600000.0
1564621200000.0
1564624800000.0
1564628400000.0
1564632000000.0
1564635600000.0
1564639200000.0
1564642800000.0
1564646400000.0
1564650000000.0
1564653600000.0
1564657200000.0
1564660800000.0
1564664400000.0
1564668000000.0
1564671600000.0
1564675200000.0
1564678800000.0
1564682400000.0
1564686000000.0
1564689600000.0
1564693200000.0
1564696800000.0
1564700400000.0
1564704000000.0
1564707600000.0
1564711200000.0
1564714800000.0
1564718400000.0
1564722000000.0
1564725600000.0
1564729200000.0
1564732800000.0
1564736400000.0
1564740000000.0
1564743600000.0
1564747200000.0
1564750800000.0
1564754400000.0
1564758000000.0
1564761600000.0
15647652

1566489600000.0
1566493200000.0
1566496800000.0
1566500400000.0
1566504000000.0
1566507600000.0
1566511200000.0
1566514800000.0
1566518400000.0
1566522000000.0
1566525600000.0
1566529200000.0
1566532800000.0
1566536400000.0
1566540000000.0
1566543600000.0
1566547200000.0
1566550800000.0
1566554400000.0
1566558000000.0
1566561600000.0
1566565200000.0
1566568800000.0
1566572400000.0
1566576000000.0
1566579600000.0
1566583200000.0
1566586800000.0
1566590400000.0
1566594000000.0
1566597600000.0
1566601200000.0
1566604800000.0
1566608400000.0
1566612000000.0
1566615600000.0
1566619200000.0
1566622800000.0
1566626400000.0
1566630000000.0
1566633600000.0
1566637200000.0
1566640800000.0
1566644400000.0
1566648000000.0
1566651600000.0
1566655200000.0
1566658800000.0
1566662400000.0
1566666000000.0
1566669600000.0
1566673200000.0
1566676800000.0
1566680400000.0
1566684000000.0
1566687600000.0
1566691200000.0
1566694800000.0
1566698400000.0
1566702000000.0
1566705600000.0
1566709200000.0
15667128

1568397600000.0
1568401200000.0
1568404800000.0
1568408400000.0
1568412000000.0
1568415600000.0
1568419200000.0
1568422800000.0
1568426400000.0
1568430000000.0
1568433600000.0
1568437200000.0
1568440800000.0
1568444400000.0
1568448000000.0
1568451600000.0
1568455200000.0
1568458800000.0
1568462400000.0
1568466000000.0
1568469600000.0
1568473200000.0
1568476800000.0
1568480400000.0
1568484000000.0
1568487600000.0
1568491200000.0
1568494800000.0
1568498400000.0
1568502000000.0
1568505600000.0
1568509200000.0
1568512800000.0
1568516400000.0
1568520000000.0
1568523600000.0
1568527200000.0
1568530800000.0
1568534400000.0
1568538000000.0
1568541600000.0
1568545200000.0
1568548800000.0
1568552400000.0
1568556000000.0
1568559600000.0
1568563200000.0
1568566800000.0
1568570400000.0
1568574000000.0
1568577600000.0
1568581200000.0
1568584800000.0
1568588400000.0
1568592000000.0
1568595600000.0
1568599200000.0
1568602800000.0
1568606400000.0
1568610000000.0
1568613600000.0
1568617200000.0
15686208

In [14]:
time_cycle_arbitrage

[1514768400000.0,
 1514772000000.0,
 1514782800000.0,
 1514790000000.0,
 1514804400000.0,
 1514811600000.0,
 1514818800000.0,
 1514833200000.0,
 1514847600000.0,
 1514854800000.0,
 1514862000000.0,
 1514872800000.0,
 1514887200000.0,
 1514894400000.0,
 1514908800000.0,
 1514912400000.0,
 1514916000000.0,
 1514919600000.0,
 1514923200000.0,
 1514926800000.0,
 1514930400000.0,
 1514948400000.0,
 1514962800000.0,
 1514988000000.0,
 1514991600000.0,
 1514998800000.0,
 1515002400000.0,
 1515006000000.0,
 1515016800000.0,
 1515020400000.0,
 1515027600000.0,
 1515052800000.0,
 1515056400000.0,
 1515067200000.0,
 1515074400000.0,
 1515078000000.0,
 1515088800000.0,
 1515092400000.0,
 1515096000000.0,
 1515106800000.0,
 1515110400000.0,
 1515117600000.0,
 1515121200000.0,
 1515124800000.0,
 1515128400000.0,
 1515132000000.0,
 1515135600000.0,
 1515139200000.0,
 1515142800000.0,
 1515157200000.0,
 1515164400000.0,
 1515168000000.0,
 1515171600000.0,
 1515175200000.0,
 1515182400000.0,
 151518600

In [15]:
ts = time_cycle_arbitrage[1]
search_ts_to_table(ts = ts)
table

[[1.0, 7.501774169591109e-05, 0.0013811200883916857, 0.12160272390101538],
 [13330.18, 1.0, 18.448142272073202, 1608.001415041245],
 [724.05, 0.054206, 1.0, 87.28212200295013],
 [8.2235, 0.00062189, 0.0114571, 1.0]]

In [32]:
USDT = coin_dict.get('USDT')
BTC = coin_dict.get('BTC')
ETH = coin_dict.get('ETH')
BNB = coin_dict.get('BNB')

In [30]:
coin_dict

{'USDT': 0, 'BTC': 1, 'ETH': 2}

In [17]:
table[USDT][BTC] * (1 - fee) * table[BTC][ETH] * (1 - fee) * table[ETH][USDT] * (1 - fee)

0.9990371726888293

In [18]:
table[USDT][ETH] * (1 - fee) * table[ETH][BTC] * (1 - fee) * table[BTC][USDT] * (1 - fee)

0.9949729671616538

In [34]:
table[USDT][BNB] * (1 - fee) * table[BNB][BTC] * (1 - fee) * table[BTC][USDT] * (1 - fee)

1.0050539046277638

In [35]:
table[USDT][BNB] * (1 - fee) * table[BNB][BTC] * (1 - fee)* table[BTC][ETH] * (1 - fee) * table[ETH][USDT] * (1 - fee) 

1.006097399981753

In [19]:
len(time_cycle_arbitrage)

1018